In [54]:
import pandas as pd
import numpy as np

In [55]:
#Estructura del dataset original
df = pd.read_stata("Mexico-2023-full-data.dta", convert_categoricals=False)
df.head()
df.shape
df.columns
df.info()
print(df.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Columns: 357 entries, idstd to a12
dtypes: float32(3), float64(165), int16(16), int32(10), int8(148), object(15)
memory usage: 2.1+ MB
['idstd', 'id', 'a4a', 'a6a', 'a2', 'a1c', 'competition_select', 'tax_select', 'vat_applicable', 'a4b_v4', 'a0', 'a3a', 'a6b', 'a6c', 'a7', 'panel', 'a1a', 'a14d', 'a14m', 'a14y', 'a14h', 'a14min', 'ic1', 'a20y', 'a20m', 'a20d', 'a1', 'a7a', 'a7b', 'a11', 'a7c', 'a9', 'b1', 'b1x', 'b3a', 'b2a', 'b2b', 'b2c', 'b2d', 'b4', 'b4a', 'b5', 'b6', 'b6b', 'b7', 'b7a', 'b8', 'b8x', 'c3', 'c4', 'c5', 'c31', 'c32', 'c6', 'c7', 'c8a', 'c8b', 'c9a', 'c9b', 'c10', 'c11', 'c12', 'c13', 'c14', 'c33', 'c34', 'c34b', 'c34bx', 'c152', 'c162', 'c172', 'c35a', 'c35b', 'c22b', 'c36', 'c37', 'c38', 'c39', 'c40a', 'c40b', 'c41a', 'c41b', 'c42', 'c30a', 'd1a1a', 'd1a1x', 'd1a6a', 'd1a6b', 'd1a6c', 'd1a6cx', 'd1a3', 'd2', 'd2x', 'd2a1', 'd2a1x', 'n3', 'n3x', 'd3a', 'd3b', 'd3c', 'd31x', 'd32', 'd4a', 'd4b',

In [56]:
#Obtencion de las variables importantes
variables = [
    'd2',              # ventas
    'l1', 'l2',        # empleados
    'b5', 'a14y',      # año inicio / año encuesta
    'd3b', 'd3c',      # exportaciones
    'e2b',             # competencia
    'c35a',            # capacitación
    'm1a_finance_pos'  # financiamiento obstáculo o facilidad de obtener financiamiento
]

df = df[variables].copy()

In [40]:
#Cambiando el nombre de las columnas
df.rename(columns={
    'd2':'sales',
    'l1':'employees_perm',
    'l2':'employees_temp',
    'b5':'year_start',
    'a14y':'interview_year',
    'd3b':'exporter_indirect',
    'd3c':'exporter_direct',
    'e2b':'competition_level',
    'c35a':'training',
    'm1a_finance_pos':'financing_obstacle'
}, inplace=True)

In [41]:
#Revisión de datos nulos normales
df.isna().sum()

sales                    0
employees_perm           0
employees_temp           0
year_start               0
interview_year           0
exporter_indirect        0
exporter_direct          0
competition_level        0
training              1277
financing_obstacle       0
dtype: int64

In [42]:
#Verificacion de datos nulos condificados segun wbes
df.replace([-9, -8, -7], np.nan, inplace=True)
df.isna().sum()

sales                    0
employees_perm           0
employees_temp          10
year_start             358
interview_year           0
exporter_indirect        0
exporter_direct          0
competition_level      452
training              1296
financing_obstacle       0
dtype: int64

In [48]:
#Eliminacion de datos nulos.
#Se imputaron los datos ya que al eliminarlos, la cantidad de registros bajan notoriamente a solo 20 empresas
df = df.dropna(subset=['year_start'])
#Imputación de datos de entrenamiento
df['training'] = df['training'].fillna(0)
#Imputación de de datos de competition level con la mediana de los registros
df['competition_level'] = df['competition_level'].fillna(
    df['competition_level'].median()
)
#Imputación de los datos de employees_Temp para convertirlos en 0
df['employees_temp'] = df['employees_temp'].fillna(0)

In [44]:
#Verificacion de datos nulos
df.isna().sum()

sales                 0
employees_perm        0
employees_temp        0
year_start            0
interview_year        0
exporter_indirect     0
exporter_direct       0
competition_level     0
training              0
financing_obstacle    0
dtype: int64

In [45]:
#Creacion de variables auxiliares
#Total de empleados
df['employees_total'] = df['employees_perm'] + df['employees_temp']
#Años totales de la empresa
df['age'] = df['interview_year'] - df['year_start']
#Variable binaria. Exportación
df['exporter'] = np.where(
    (df['exporter_direct'] > 0) | 
    (df['exporter_indirect'] > 0), 1, 0
)
#Productividad laboral
df['labor_productivity'] = df['sales'] / df['employees_total']

In [46]:
#Consistencia de los datos
df.describe()
df.isna().sum()
df[df['age'] < 0]
df[df['sales'] <= 0]
print(df[df['employees_total'] <= 0])

Empty DataFrame
Columns: [sales, employees_perm, employees_temp, year_start, interview_year, exporter_indirect, exporter_direct, competition_level, training, financing_obstacle, employees_total, age, exporter, labor_productivity]
Index: []


In [51]:
#Observacion del nuevo dataset
df.head()
df.shape
df.columns
df.info()
print(df.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
Index: 964 entries, 1 to 1320
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sales               964 non-null    int32  
 1   employees_perm      964 non-null    int16  
 2   employees_temp      964 non-null    float64
 3   year_start          964 non-null    float64
 4   interview_year      964 non-null    int16  
 5   exporter_indirect   964 non-null    int8   
 6   exporter_direct     964 non-null    int8   
 7   competition_level   964 non-null    float64
 8   training            964 non-null    float64
 9   financing_obstacle  964 non-null    int8   
 10  employees_total     964 non-null    float64
 11  age                 964 non-null    float64
 12  exporter            964 non-null    int64  
 13  labor_productivity  964 non-null    float64
dtypes: float64(7), int16(2), int32(1), int64(1), int8(3)
memory usage: 78.1 KB
['sales', 'employees_perm', 'employ

In [50]:
#Registros del dataset
df.describe()

,sales,employees_perm,employees_temp,year_start,interview_year,exporter_indirect,exporter_direct,competition_level,training,financing_obstacle,employees_total,age,exporter,labor_productivity
count,9.640000e+02,964.000000,964.000000,964.000000,964.0,964.000000,964.000000,964.000000,964.000000,964.000000,964.000000,964.000000,964.000000,9.640000e+02
mean,9.863234e+07,87.689834,81.206432,1998.918050,2023.0,2.282158,1.934647,-1.975104,0.034232,8.578838,168.896266,24.081950,0.150415,5.486886e+05
std,1.508862e+08,115.061996,110.085164,18.848539,0.0,7.329173,8.373231,3.319817,0.429772,4.421111,224.840263,18.848539,0.357663,3.531601e+05
min,3.800000e+05,5.000000,0.000000,1833.000000,2023.0,0.000000,0.000000,-4.000000,0.000000,1.000000,5.000000,3.000000,0.000000,3.800000e+04
25%,1.000000e+07,12.000000,10.000000,1993.000000,2023.0,0.000000,0.000000,-4.000000,0.000000,4.000000,23.000000,13.000000,0.000000,3.312818e+05
50%,3.100000e+07,32.000000,28.500000,2004.000000,2023.0,0.000000,0.000000,-4.000000,0.000000,9.000000,60.000000,19.000000,0.000000,4.922691e+05
75%,1.300000e+08,142.000000,128.500000,2010.000000,2023.0,0.000000,0.000000,2.000000,0.000000,12.000000,270.000000,30.000000,0.000000,6.722859e+05
max,1.400000e+09,1680.000000,1650.000000,2020.000000,2023.0,50.000000,70.000000,15.000000,10.000000,15.000000,3330.000000,190.000000,1.000000,2.787879e+06


In [52]:
#Creacion del nuevo dataset
df.reset_index(drop=True, inplace=True)
df.to_csv("Mexico_2023_clean.csv", index=False)